In [1]:
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt
import numpy as np
from scipy.spatial import ConvexHull
import math
import autograd.numpy as np
from autograd import grad
import random
import cvxpy as cp
import time
import csv
import pandas as pd
from scipy.spatial import distance
import sympy as sp

In [3]:
import PlatonicCoordinates as pcoord
import ArchimedeanCoordinates as acoord
import CatalanCoordinates as ccoord
import JohnsonCoordinates as jcoord

# Polyhedron Movement

In [4]:
# Either in 2D or 3D translate each set of coordinates by a vector of translations
def Translate2D(pts, delta):
    for row in pts:
        row += delta
    return pts
    
    
def Translate3D(pts, delta):
    for row in pts:
        row += delta
    return pts

In [5]:
# Take in a vector of translations, using the below translation matrix
# Rotating counterclockwise each way
def TranslationMatrix(x): 
    return np.array([[np.cos(x), -np.sin(x)], [np.sin(x), np.cos(x)]])

def Rotate2D(pts, theta):
    newPts = []
    for row in pts:
        transMatrix = TranslationMatrix(theta)
        row = np.matmul(TranslationMatrix(theta), row)
        newPts.append(row)
    return np.array(newPts)
    

def Rotate3D(pts, theta):
    newPts = []
    for row in pts:
        xRotated = np.matmul(TranslationMatrix(theta[0]), row[1:3]) 
        newRow1 = np.array([row[0], xRotated[0], xRotated[1]])

        yRotated = np.matmul(TranslationMatrix(theta[1]),newRow1[[2,0]])
        newRow2 = np.array([yRotated[1], newRow1[1], yRotated[0]])
    
        zRotated = np.matmul(TranslationMatrix(theta[2]), newRow2[0:2])
        newRow3 = np.array([zRotated[0], zRotated[1], newRow2[2]])
        
        newPts.append(newRow3)

    return np.array(newPts)

In [6]:
# Rescale a polyhedron to be mu times larger.
# Returning a new list of points for the resulting 2D or 3D polyhedron
def Rescale2D(pts, mu):
    pts *= mu
    return pts
    
def Rescale3D(pts, mu):
    pts *= mu
    return pts

In [7]:
# Project a 3D polyhedron down onto its shadow (dropping the z component)
# Returning a new list of points for the resulting 2D polyhedron
def Projection(pts):
    newpts = pts[:, :-1]
    return newpts

# Passage Checking

In [8]:
# Use the Graham scan to find the convex hull
def GrahamScan(points):
    def CrossProduct(p1, p2, p3):
        return (p2[0] - p1[0]) * (p3[1] - p1[1]) - (p2[1] - p1[1]) * (p3[0] - p1[0])

    def AngleWithPivot(point):
        return math.atan2(point[1] - pivot[1], point[0] - pivot[0])

    pivot = min(points, key=lambda point: (point[1], point[0]))

    sorted_points = sorted(points, key=AngleWithPivot)

    convex_hull = [pivot, sorted_points[0]]

    for i in range(1, len(sorted_points)):
        while (
            len(convex_hull) > 1
            and CrossProduct(convex_hull[-2], convex_hull[-1], sorted_points[i]) <= 0
        ):
            convex_hull.pop()
        convex_hull.append(sorted_points[i])

    return convex_hull

In [9]:
# Compute the largest mu such that rescaling the first given 2D polyhedron lies entirely inside the second given polyhedron.
def LargestContainment(pts_inner, pts_outer):
    innerProj = Projection(pts_inner)
    outerProj = Projection(pts_outer)
    innerHull = GrahamScan(innerProj)
    outerHull = GrahamScan(outerProj)
    
    store = 1000
    # Go through each point in our outer hull, making sure they're not the same point
    for i in range(len(outerHull)):
        p1 = outerHull[i]
        p2 = outerHull[(i+1)%len(outerHull)]
        if p1[0] == p2[0] and p1[1] == p2[1]:
            continue
        matrix = np.matrix([[p1[0], p1[1]], [p2[0], p2[1]]])
        matrixInverse = np.linalg.inv(matrix)
        b = ([[1], [1]])
        product = np.matmul(matrixInverse, b)
        # This tells us the line that defines our outer point
        a1 = product[0,0]
        a2 = product[1,0]
        # Assumes that only the points on the boundary would matter 
        # Val tells us where we are with our point in relation to the line
        for j in range(len(innerHull)):
            o1 = innerHull[j]
            val = a1*o1[0] + a2*o1[1]
            if val > 0 and 1/val < store:
                store = (1/val)
    return store

In [10]:
# For a set of rotations and translations return how much we can rescale by
def MaxShapeMu(delta1, delta2, theta11, theta12, theta13, theta21, theta22, theta23, P):
    delta_set = np.array([delta1, delta2, 0])
    theta1_set = np.array([theta11, theta12, theta13])
    theta2_set = np.array([theta21, theta22, theta23])
    P1 = Rotate3D(P, theta1_set)
    P2 = Translate3D(P1, delta_set)
    P3 = Rotate3D(P, theta2_set)
    return LargestContainment(P2, P3)

# Gradient Setup and Evaluation

In [11]:
# For each trio of inner point and outer edge, symbolically compute the gradient and store it in a matrix to return
def GradientVals(a1, a2, a3, a4, a5, a6, a7, a8, trios, P):
    
    grad_mat = []
    
    for trio in trios:
        # Symbolic constants
        v1, v2, v3, v4, v5, v6, v7, v8 = sp.symbols('v1 v2 v3 v4 v5 v6 v7 v8')
        
        P1 = P[trio[0]]
        P2 = P[trio[1]]
        P3 = P[trio[2]]
        # Our three points
        x1, y1, z1 = P1
        x2, y2, z2 = P2
        x3, y3, z3 = P3
        
        bigP = []
        bigP.append(P1)
        bigP.append(P2)
        bigP.append(P3)
        
        # Set up what our expression looks like after rotations and translations
        x1new = (sp.cos(v4)*sp.cos(v5))*x1 + (sp.cos(v5)*sp.sin(v3)*sp.sin(v4) - sp.cos(v3)*sp.sin(v5))*y1 + (sp.sin(v3)*sp.sin(v5) + sp.cos(v3)*sp.cos(v5)*sp.sin(v4))*z1 + v1
        y1new = (sp.cos(v4)*sp.sin(v5))*x1 + (sp.cos(v3)*sp.cos(v5) + sp.sin(v3)*sp.sin(v4)*sp.sin(v5))*y1 + (sp.cos(v3)*sp.sin(v4)*sp.sin(v5) - sp.cos(v5)*sp.sin(v3))*z1 + v2
        
        x2new = (sp.cos(v7)*sp.cos(v8))*x2 + (sp.cos(v8)*sp.sin(v6)*sp.sin(v7) - sp.cos(v6)*sp.sin(v8))*y2 + (sp.sin(v6)*sp.sin(v8) + sp.cos(v6)*sp.cos(v8)*sp.sin(v7))*z2
        y2new = (sp.cos(v7)*sp.sin(v8))*x2 + (sp.cos(v6)*sp.cos(v8) + sp.sin(v6)*sp.sin(v7)*sp.sin(v8))*y2 + (sp.cos(v6)*sp.sin(v7)*sp.sin(v8) - sp.cos(v8)*sp.sin(v6))*z2
        
        x3new = (sp.cos(v7)*sp.cos(v8))*x3 + (sp.cos(v8)*sp.sin(v6)*sp.sin(v7) - sp.cos(v6)*sp.sin(v8))*y3 + (sp.sin(v6)*sp.sin(v8) + sp.cos(v6)*sp.cos(v8)*sp.sin(v7))*z3
        y3new = (sp.cos(v7)*sp.sin(v8))*x3 + (sp.cos(v6)*sp.cos(v8) + sp.sin(v6)*sp.sin(v7)*sp.sin(v8))*y3 + (sp.cos(v6)*sp.sin(v7)*sp.sin(v8) - sp.cos(v8)*sp.sin(v6))*z3

        
        expression = (x2new*y3new - x3new*y2new) / ((y3new - y2new)*x1new + (x2new - x3new)*y1new)
        
        variables = [v1, v2, v3, v4, v5, v6, v7, v8]
        # Take the gradient with respect to each variable and evaluate those gradients
        gradients = [sp.diff(expression, var) for var in variables]
    
        eval_gradients = [grad.evalf(subs={v1: a1, v2: a2, v3: a3, v4: a4, v5: a5, v6: a6, v7: a7, v8: a8}) for grad in gradients]
        
        eval_gradients = np.array(eval_gradients, dtype=float)
        # Store our gradients in a matrix to be returned
        grad_mat.append(eval_gradients)
        
    return grad_mat

In [12]:
# Go through our list of trios (inner point and outer edge) and find the largest
# These are the trios whose gradients we will look at
def MuVals(delta1, delta2, theta11, theta12, theta13, theta21, theta22, theta23, trios, P):
    notable_trios = []
    trio_vals = {}
    base = 10
    
    for trio in trios:
        P1 = P[trio[0]]
        P2 = P[trio[1]]
        P3 = P[trio[2]]
        # We take in where the points are respectively (i.e. 4th inner point)
        # So we must rotate and translate each point in our trio again
        delta_set = np.array([delta1, delta2, 0])
        theta1_set = np.array([theta11, theta12, theta13])
        theta2_set = np.array([theta21, theta22, theta23])
                
        x1, y1 = Projection(Translate3D((Rotate3D([P1], theta1_set)), delta_set))[0]
        x2, y2 = Projection(Rotate3D([P2], theta2_set))[0]
        x3, y3 = Projection(Rotate3D([P3], theta2_set))[0]
        
        muVal = (x2*y3 - x3*y2) / ((y3-y2)*x1 + (x2-x3)*y1)
        
        trio_key = tuple(trio)
        
        trio_vals[trio_key] = muVal
        # If we have a higher muVal store it
        if muVal < base and muVal > 0:
            base = muVal
    # Send us all of the notable trios
    for key in trio_vals:
        if trio_vals[key] - base < 0.0001 and trio_vals[key] > 0:
            notable_trios.append(key)
    
    return [notable_trios]

In [13]:
# Find all inner point and outside edge combinations 
def ConvexHullTrios(delta1, delta2, theta11, theta12, theta13, theta21, theta22, theta23, P):
    
    delta_set = np.array([delta1, delta2, 0])
    theta1_set = np.array([theta11, theta12, theta13])
    theta2_set = np.array([theta21, theta22, theta23])
    P1 = Rotate3D(P, theta1_set)
    P2 = Translate3D(P1, delta_set) 
    P3 = Rotate3D(P, theta2_set) 
    
    # Use rotations and translations to find the inner and outer points and hulls
    # Iterate through them and find the trios to search through
    innerProj = Projection(P2)
    outerProj = Projection(P3)
    innerHull = GrahamScan(innerProj)
    outerHull = GrahamScan(outerProj)
    
    innerList = []
    
    for point in innerHull:
        for i in range(len(innerProj)):
            if np.array_equal(point, innerProj[i]):
                innerList.append(i)
                
    outerList = []
    
    for point in outerHull:
        for i in range(len(outerProj)):
            if np.array_equal(point, outerProj[i]):
                outerList.append(i)
                
    pairings = []
    # Our pairs are all of the inner points combined with an outer edge 
    for i in range(len(innerList)):
        for j in range(len(outerList)):
            pairings.append([innerList[i], outerList[j], outerList[(j+1)%len(outerList)]])
            
            
    return pairings

# Shape Testing 

In [14]:
# Given a shape this will start a random initailization and use gradient ascent to find the largest rescaling 
def TestShapeNew(shape):
    
    # Initialize a random set of rotations and translations    
    step = 0.00001
    x1 = np.random.uniform(-0.01,0.01)
    x2 = np.random.uniform(-0.01,0.01)

    x3 = np.random.uniform(0,2 * np.pi)
    x4 = np.random.uniform(0,2 * np.pi)
    x5 = np.random.uniform(0,2 * np.pi)

    x6 = np.random.uniform(0,2 * np.pi)
    x7 = np.random.uniform(0,2 * np.pi)
    x8 = np.random.uniform(0,2 * np.pi)

    a=1.0
    
    # Iterate a max of 1000 times
    for j in range(1000):
        grad_mat = []
        muVal = 0
        # Find our trios, figure out which ones matter, and evaluate their gradients
        trios = ConvexHullTrios(x1, x2, x3, x4, x5, x6, x7, x8, shape)
        notable_trios = MuVals(x1, x2, x3, x4, x5, x6, x7, x8, trios, shape)[0]
        grad_mat = GradientVals(x1, x2, x3, x4, x5, x6, x7, x8, notable_trios, shape)
        # If there is only one active gradient this one will do for us
        avgGrads = grad_mat[0] 

        # If there are many active gradients we want the combination with smallest norm
        if len(grad_mat)>1: 
            grad_mat = np.array(grad_mat)
            # Objective: Minimize norm of convex combination
            # For minimum norm, we minimize (1/2) * y.T * (A * A.T) * y
            y = cp.Variable(len(grad_mat))
            objective = cp.Minimize((1/2) * cp.norm(grad_mat.T @ y, 2))

            # Constraints: weights sum up to 1 and are all non-negative
            constraints = [cp.sum(y) == 1, y >= 0]
            # Problem setup
            problem = cp.Problem(objective, constraints)
            # Solve the problem
            problem.solve()
            # The optimal weights for the minimum norm convex combination
            avgGrads = np.matmul(np.transpose(grad_mat), y.value)
            
        # Use backtracking to scale the gradients
        a = a*2 
        for i in range(20):
            temp1 = x1 + a*avgGrads[0]
            temp2 = x2 + a*avgGrads[1]
            temp3 = x3 + a*avgGrads[2]
            temp4 = x4 + a*avgGrads[3]
            temp5 = x5 + a*avgGrads[4]
            temp6 = x6 + a*avgGrads[5]
            temp7 = x7 + a*avgGrads[6]
            temp8 = x8 + a*avgGrads[7]
            if MaxShapeMu(temp1, temp2, temp3, temp4, temp5, temp6, temp7, temp8, shape) > MaxShapeMu(x1, x2, x3, x4, x5, x6, x7, x8, shape): 
                break
            else:
                a *= 0.5 

        # Update our rotations and translations
        x1 += avgGrads[0]* a
        x2 += avgGrads[1]* a
        x3 += avgGrads[2]* a
        x4 += avgGrads[3]* a
        x5 += avgGrads[4]* a
        x6 += avgGrads[5]* a
        x7 += avgGrads[6]* a
        x8 += avgGrads[7]* a
        # Note that our final vector is given with 8 parameters but can be converted to the 7 referenced
        # By subracting x8 from both x5 and x8
        mu = MaxShapeMu(x1, x2, x3, x4, x5, x6, x7, x8, shape)
        final_values = [x1, x2, x3, x4, x5, x6, x7, x8]
        # If our a is small enough we won't see much change so we end
        if a < 1e-8:
            print(j)
            break
    return [mu, final_values]


In [15]:
# For a set amount of time (in seconds) this will repeatedly run our method on a certain shape
# Floor is the highest previously observed mu value
# Returns the number of runs, number bigger than previous, number bigger than one (successful passages)
# The worst and best rescalings, and the coordinates of the best rescaling
def TimedShapeRepeat(runtime, shape, floor):
    desired_run_time = runtime
    start_time = time.time()
    
    gd_runs = 0
    num_bigger_prev = 0
    num_bigger_one = 0
    mu_min = 2
    mu_max = 0
    x_best = []
    
    
    while True:
        result = TestShapeNew(shape)
        
        gd_runs += 1
        
        if result[0] >= 1:
            num_bigger_one += 1
            
        if result[0] >= floor:
            num_bigger_prev += 1
            
        if result[0] > mu_max:
            mu_max = result[0]
            x_best = result[1]
        
        if result[0] < mu_min: 
            mu_min = result[0]
            
        
        if time.time() - start_time > desired_run_time:
            return [gd_runs, num_bigger_prev, num_bigger_one, mu_min, mu_max, x_best]

# Example Platonic Solid Execution

In [ ]:
tetrahedron = pcoord.Tetrahedron()
TestShapeNew(tetrahedron)

# Example Archimedean Solid Execution

In [ ]:
rhombicosidodecahedron = acoord.Rhombicosidodecahedron()
TestShapeNew(rhombicosidodecahedron)

# Example Catalan Solid Execution

In [ ]:
triakis_tetrahedron = ccoord.TriakisTetrahedron()
TestShapeNew(triakis_tetrahedron)

# Example Johnson Solid Execution

In [ ]:
bigyrate_diminishsed_rhombicosidodecahedron = jcoord.BigyrateDiminishedRhombicosidodecahedron()
TestShapeNew(bigyrate_diminishsed_rhombicosidodecahedron)